# Part 2 - Rainfall input

In this notebook demonstrates how the rainfall data is extracted from RADKLIM data as time series for every subbasins and how the counterfactuals are created. In the second part we calculate the effective rainfall time series.
For this example we supply the RADKLIM data for the event NW/Jul21 which has been downloaded from the RADKLIM data from the DWD Open Data Server (ref). The files were then compiled using the package radolan_to_netcdf (ref).

We supply following data:
- **nw_jul21.nc**: NetCDF containing the RADKLIM precipitation estimates for the NW/JUL21 event.

We then compute the precipitation time series for all subbasins for the NW/Jul21 event and create counterfactuals by moving the centroid (cell with the highest hourly rainfall) to every subbasin in the data set.

This script is based on the tutorial on the wradlib homepage: https://docs.wradlib.org/en/latest/notebooks/workflow/recipe5.html. Unlike other packages the extraction of rainfall series for subbasins is quite precise. With the 1 km grid of RADKLIM some cells might cover a subbasin just a little bit. The wradlib algorithm computes the weighted areal average, while most other methods either use the average of all fully included cells, or the average of all cells with an overlap.

As a second step we also calculate the effective precipitation from the RADKLIM precipitation time series.

In [1]:
import pandas as pd
import counterfactual_precipitation as cp
import geopandas as gpd
import pyproj
from osgeo import osr
import wradlib as wrl
import os
import xarray as xr
import numpy as np

In [3]:
# create the files for the projection information
radolanwkt = """PROJCS["Radolan Projection",
    GEOGCS["Radolan Coordinate System",
        DATUM["Radolan_Kugel",
            SPHEROID["Erdkugel",6370040,0]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]]],
    PROJECTION["Polar_Stereographic"],
    PARAMETER["latitude_of_origin",90],
    PARAMETER["central_meridian",10],
    PARAMETER["scale_factor",0.933012701892],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["kilometre",1,
        AUTHORITY["EPSG","9036"]],
    AXIS["Easting",SOUTH],
    AXIS["Northing",SOUTH]]
"""
radolanproj = pyproj.CRS.from_wkt(radolanwkt)
proj_stereo = wrl.georef.create_osr("dwd-radolan-sphere")

# This is our target projection (UTM)
proj_utm = osr.SpatialReference()
proj_utm.ImportFromEPSG(32632)


event_id = "WG/Jul21"
subbasins = gpd.read_file("output/gis/subbasins_info.gpkg")
#project the subbasin shapefile to UTM
subbasins_utm = subbasins.to_crs("EPSG:32632")
subbasins_utm = subbasins_utm[["sub_id", "geometry"]]
subbasins_utm.to_file("output/gis/subbasins_utm.gpkg")

subbasins = subbasins.to_crs(radolanproj)

event = xr.open_dataset("input/nw_jul21.nc")
event = event.rainfall_amount

#get the coordinates of the centroid
t_max, y_max, x_max = np.argwhere(event.values == np.nanmax(event.values))[0]
centroid_x_coord = event.x[x_max].item()
centroid_y_coord = event.y[y_max].item()
original_centroid = (centroid_x_coord, centroid_y_coord)

if not os.path.exists("output/precipitation"):
    os.mkdir("output/precipitation")

## Extract precipitation time series for the original event (-999) and all counterfactuals
For this we use the same function as below but we manipulate the subbasin shapefile and add one basin, which has the same centroid as the centroid of the original event. The subbasin ID -999 desribes the event in its original spatial position.

In [4]:
original_event = {"sub_id": -999, "geometry": subbasins.geometry[0]} #the geometry column won't be used, so we just copy another geometry
original_event = gpd.GeoDataFrame([original_event], geometry='geometry')
original_event.set_crs(radolanproj, inplace=True)

subbasins = gpd.GeoDataFrame(pd.concat([subbasins, original_event], ignore_index=True))

if not os.path.exists("output/basins/-999"):
    os.mkdir("output/basins/-999")

In [ ]:
for id in subbasins.sub_id.to_list():
    cp.get_rainseries_from_radklim(subbasin_id=id,
                               event_id="nw_jul21",
                               subbasins_radolanproj=subbasins,
                               original_event=event,
                               original_centroid=original_centroid,
                               radolan_projection=radolanproj,
                               utm_projection=proj_utm,
                               stereo_projection=proj_stereo
                               )


## Calculate effective precipitation
We now transform the RADKLIM precipitation time series into effective precipitation. The curve numbers of the basins are stored in **output/gis/CN_subbasins_table.csv**

In [2]:
if not os.path.exists('output/effective_rainfall'):
    os.mkdir('output/effective_rainfall')

subbasins = gpd.read_file("output/gis/subbasins_info.gpkg")

for id in subbasins.sub_id.to_list():
    cp.get_effective_rainfall(id)

Sub ID 1
Sub ID 2
Sub ID 3
Sub ID 4
Sub ID 5
Sub ID 6
Sub ID 7
Sub ID 8
Sub ID 9
Sub ID 10
Sub ID 11
Sub ID 12
Sub ID 13
Sub ID 14
Sub ID 15
Sub ID 16
Sub ID 17
Sub ID 18
Sub ID 19
Sub ID 20
Sub ID 21
Sub ID 22
Sub ID 23
Sub ID 24
Sub ID 25
Sub ID 26
Sub ID 27
Sub ID 28
Sub ID 29
Sub ID 30
Sub ID 31
Sub ID 32
Sub ID 33
Sub ID 34
Sub ID 35
Sub ID 36
Sub ID 37
Sub ID 38
Sub ID 39
Sub ID 40
Sub ID 41
Sub ID 42
Sub ID 43
Sub ID 44
Sub ID 45
Sub ID 46
Sub ID 47
Sub ID 48
Sub ID 49
Sub ID 50
Sub ID 51
Sub ID 52
Sub ID 53
Sub ID 54
Sub ID 55
Sub ID 56


If all these steps ran succesfully, we now have generated all input data necessary for the modelling of quick runoff. Proceed with Part 3